## Basic experiment tools dev notebook

Created on: Tuesday March 22nd, 2022  
Created by: Jacob Alexander Rose  

In [1]:
%load_ext autoreload
%autoreload 2

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
from pathlib import Path
from icecream import ic
from rich import print as pp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# from imutils.big.datamodule import Herbarium2022DataModule, Herbarium2022Dataset
from imutils.ml.data.datamodule import Herbarium2022DataModule, Herbarium2022Dataset
from imutils.ml.utils.etl_utils import ETL

import pytorch_lightning as pl
from torchvision import transforms as T
import argparse
import imutils

from hydra.experimental import compose, initialize, initialize_config_dir
import hydra

from omegaconf import DictConfig, OmegaConf
from typing import *

Importing imutils
ASSETS_DIR: /media/data/jacob/GitHub/image-utils/assets
SAMPLE_IMAGE_PATHS: [PosixPath('/media/data/jacob/GitHub/image-utils/assets/Ericaceae_Zenobia_pulverulenta_7984 {WolfeUSGS} [1.96x].jpg'), PosixPath('/media/data/jacob/GitHub/image-utils/assets/Ericaceae_Arbutus_densiflora_1440 {WolfeUSGS} [1.96x].jpg')]
IMUTILS_ML_ROOT: /media/data/jacob/GitHub/image-utils/imutils/ml


[
    'BASE_ML_CONF_PATH',
    'IMUTILS_ML_ROOT',
    '__builtins__',
    '__cached__',
    '__doc__',
    '__file__',
    '__loader__',
    '__name__',
    '__package__',
    '__path__',
    '__spec__',
    'os',
    'pp'
]

### helper display func

In [2]:
def display_train_timing_info(batches_per_epoch: int,
                              batches_per_second: float,
                              batch_size: int):
    
    samples_per_epoch = batches_per_epoch*batch_size
    seconds_per_epoch = batches_per_epoch * batches_per_second
    min_per_epoch = seconds_per_epoch / 60
    hrs_per_epoch = min_per_epoch / 60
    
    samples_per_second = batches_per_second * batch_size

    batches_per_min = batches_per_second * 60
    batches_per_hr = batches_per_min * 60
    
    samples_per_min = samples_per_second * 60
    samples_per_hr = samples_per_min * 60


    pp([f"seconds_per_epoch: {seconds_per_epoch:>,}",
       f"min_per_epoch: {min_per_epoch:.4f}",
       f"hrs_per_epoch: {hrs_per_epoch:.4f}",
       f"epochs_per_second: {1/seconds_per_epoch:.4f}",
       f"epochs_per_min: {1/min_per_epoch:.4f}",
       f"epochs_per_hr: {1/hrs_per_epoch:.4f}",
       f"batches_per_epoch: {batches_per_epoch:.4g}",
       f"samples_per_epoch: {samples_per_epoch:.4g}",
       f"seconds_per_batch: {1/batches_per_second:.4f}",
       f"batches_per_second: {batches_per_second:.4f}",
       f"batches_per_min: {batches_per_min:.4f}",
       f"batches_per_hr: {batches_per_hr:.4f}",
       f"samples_per_second: {samples_per_second:.4f}",
       f"samples_per_min: {samples_per_min:.4f}",
       f"samples_per_hr: {samples_per_hr:.4g}",
       f"batch_size: {batch_size}"])

### Experiment #2


In [3]:
batches_per_second = (1/1.7)
batches_per_epoch = 4374
batch_size=48

print(f"Experiment #2: batch_size={batch_size}, num_processes=4, num_devices=2")
print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #2: batch_size=48, num_processes=4, num_devices=2
Using 50% of samples


[
    'seconds_per_epoch: 2,572.9411764705883',
    'min_per_epoch: 42.8824',
    'hrs_per_epoch: 0.7147',
    'epochs_per_second: 0.0004',
    'epochs_per_min: 0.0233',
    'epochs_per_hr: 1.3992',
    'batches_per_epoch: 4374',
    'samples_per_epoch: 2.1e+05',
    'seconds_per_batch: 1.7000',
    'batches_per_second: 0.5882',
    'batches_per_min: 35.2941',
    'batches_per_hr: 2117.6471',
    'samples_per_second: 28.2353',
    'samples_per_min: 1694.1176',
    'samples_per_hr: 1.016e+05',
    'batch_size: 48'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 5,145.882352941177',
    'min_per_epoch: 85.7647',
    'hrs_per_epoch: 1.4294',
    'epochs_per_second: 0.0002',
    'epochs_per_min: 0.0117',
    'epochs_per_hr: 0.6996',
    'batches_per_epoch: 8748',
    'samples_per_epoch: 4.199e+05',
    'seconds_per_batch: 1.7000',
    'batches_per_second: 0.5882',
    'batches_per_min: 35.2941',
    'batches_per_hr: 2117.6471',
    'samples_per_second: 28.2353',
    'samples_per_min: 1694.1176',
    'samples_per_hr: 1.016e+05',
    'batch_size: 48'
]

### Experiment #3

In [4]:
batches_per_second = (1/2.15)
batches_per_epoch = 3280
batch_size=64
print(f"Experiment #3: batch_size=64, num_processes=4, num_devices=2")
print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #3: batch_size=64, num_processes=4, num_devices=2
Using 50% of samples


[
    'seconds_per_epoch: 1,525.581395348837',
    'min_per_epoch: 25.4264',
    'hrs_per_epoch: 0.4238',
    'epochs_per_second: 0.0007',
    'epochs_per_min: 0.0393',
    'epochs_per_hr: 2.3598',
    'batches_per_epoch: 3280',
    'samples_per_epoch: 2.099e+05',
    'seconds_per_batch: 2.1500',
    'batches_per_second: 0.4651',
    'batches_per_min: 27.9070',
    'batches_per_hr: 1674.4186',
    'samples_per_second: 29.7674',
    'samples_per_min: 1786.0465',
    'samples_per_hr: 1.072e+05',
    'batch_size: 64'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 3,051.162790697674',
    'min_per_epoch: 50.8527',
    'hrs_per_epoch: 0.8475',
    'epochs_per_second: 0.0003',
    'epochs_per_min: 0.0197',
    'epochs_per_hr: 1.1799',
    'batches_per_epoch: 6560',
    'samples_per_epoch: 4.198e+05',
    'seconds_per_batch: 2.1500',
    'batches_per_second: 0.4651',
    'batches_per_min: 27.9070',
    'batches_per_hr: 1674.4186',
    'samples_per_second: 29.7674',
    'samples_per_min: 1786.0465',
    'samples_per_hr: 1.072e+05',
    'batch_size: 64'
]

### Experiment #4

In [5]:
batches_per_second = (1/3.3)
batches_per_epoch = 2187
batch_size=96

print(f"Experiment #4: batch_size={batch_size}, num_processes=4, num_devices=2")
print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #4: batch_size=96, num_processes=4, num_devices=2
Using 50% of samples


[
    'seconds_per_epoch: 662.7272727272727',
    'min_per_epoch: 11.0455',
    'hrs_per_epoch: 0.1841',
    'epochs_per_second: 0.0015',
    'epochs_per_min: 0.0905',
    'epochs_per_hr: 5.4321',
    'batches_per_epoch: 2187',
    'samples_per_epoch: 2.1e+05',
    'seconds_per_batch: 3.3000',
    'batches_per_second: 0.3030',
    'batches_per_min: 18.1818',
    'batches_per_hr: 1090.9091',
    'samples_per_second: 29.0909',
    'samples_per_min: 1745.4545',
    'samples_per_hr: 1.047e+05',
    'batch_size: 96'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 1,325.4545454545455',
    'min_per_epoch: 22.0909',
    'hrs_per_epoch: 0.3682',
    'epochs_per_second: 0.0008',
    'epochs_per_min: 0.0453',
    'epochs_per_hr: 2.7160',
    'batches_per_epoch: 4374',
    'samples_per_epoch: 4.199e+05',
    'seconds_per_batch: 3.3000',
    'batches_per_second: 0.3030',
    'batches_per_min: 18.1818',
    'batches_per_hr: 1090.9091',
    'samples_per_second: 29.0909',
    'samples_per_min: 1745.4545',
    'samples_per_hr: 1.047e+05',
    'batch_size: 96'
]

### Experiment #5

Started: 3:15 AM - 2022-03-23  
Ended:   4:30 AM - 2022-03-23  

In [6]:
batches_per_second = (1/4.3)
batches_per_epoch = 1640
batch_size=128

print(f"Experiment #5: batch_size={batch_size}, num_processes=4, num_devices=2")
print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #5: batch_size=128, num_processes=4, num_devices=2
Using 50% of samples


[
    'seconds_per_epoch: 381.39534883720927',
    'min_per_epoch: 6.3566',
    'hrs_per_epoch: 0.1059',
    'epochs_per_second: 0.0026',
    'epochs_per_min: 0.1573',
    'epochs_per_hr: 9.4390',
    'batches_per_epoch: 1640',
    'samples_per_epoch: 2.099e+05',
    'seconds_per_batch: 4.3000',
    'batches_per_second: 0.2326',
    'batches_per_min: 13.9535',
    'batches_per_hr: 837.2093',
    'samples_per_second: 29.7674',
    'samples_per_min: 1786.0465',
    'samples_per_hr: 1.072e+05',
    'batch_size: 128'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 762.7906976744185',
    'min_per_epoch: 12.7132',
    'hrs_per_epoch: 0.2119',
    'epochs_per_second: 0.0013',
    'epochs_per_min: 0.0787',
    'epochs_per_hr: 4.7195',
    'batches_per_epoch: 3280',
    'samples_per_epoch: 4.198e+05',
    'seconds_per_batch: 4.3000',
    'batches_per_second: 0.2326',
    'batches_per_min: 13.9535',
    'batches_per_hr: 837.2093',
    'samples_per_second: 29.7674',
    'samples_per_min: 1786.0465',
    'samples_per_hr: 1.072e+05',
    'batch_size: 128'
]

### Experiment #6

Started: 4:30 AM - 2022-03-23  
Ended:   5:45 AM - 2022-03-23  

In [7]:
batches_per_second = (1/5.15)
batches_per_epoch = 1458
batch_size=144

print(f"Experiment #6: batch_size={batch_size}, num_processes=4, num_devices=2")
print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #6: batch_size=144, num_processes=4, num_devices=2
Using 50% of samples


[
    'seconds_per_epoch: 283.1067961165048',
    'min_per_epoch: 4.7184',
    'hrs_per_epoch: 0.0786',
    'epochs_per_second: 0.0035',
    'epochs_per_min: 0.2119',
    'epochs_per_hr: 12.7160',
    'batches_per_epoch: 1458',
    'samples_per_epoch: 2.1e+05',
    'seconds_per_batch: 5.1500',
    'batches_per_second: 0.1942',
    'batches_per_min: 11.6505',
    'batches_per_hr: 699.0291',
    'samples_per_second: 27.9612',
    'samples_per_min: 1677.6699',
    'samples_per_hr: 1.007e+05',
    'batch_size: 144'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 566.2135922330096',
    'min_per_epoch: 9.4369',
    'hrs_per_epoch: 0.1573',
    'epochs_per_second: 0.0018',
    'epochs_per_min: 0.1060',
    'epochs_per_hr: 6.3580',
    'batches_per_epoch: 2916',
    'samples_per_epoch: 4.199e+05',
    'seconds_per_batch: 5.1500',
    'batches_per_second: 0.1942',
    'batches_per_min: 11.6505',
    'batches_per_hr: 699.0291',
    'samples_per_second: 27.9612',
    'samples_per_min: 1677.6699',
    'samples_per_hr: 1.007e+05',
    'batch_size: 144'
]

In [8]:
batches_per_second = (1/4.87)
batches_per_epoch = 1458
batch_size=144

print(f"Experiment #6: batch_size={batch_size}, num_processes=4, num_devices=2")
print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #6: batch_size=144, num_processes=4, num_devices=2
Using 50% of samples


[
    'seconds_per_epoch: 299.38398357289526',
    'min_per_epoch: 4.9897',
    'hrs_per_epoch: 0.0832',
    'epochs_per_second: 0.0033',
    'epochs_per_min: 0.2004',
    'epochs_per_hr: 12.0247',
    'batches_per_epoch: 1458',
    'samples_per_epoch: 2.1e+05',
    'seconds_per_batch: 4.8700',
    'batches_per_second: 0.2053',
    'batches_per_min: 12.3203',
    'batches_per_hr: 739.2197',
    'samples_per_second: 29.5688',
    'samples_per_min: 1774.1273',
    'samples_per_hr: 1.064e+05',
    'batch_size: 144'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 598.7679671457905',
    'min_per_epoch: 9.9795',
    'hrs_per_epoch: 0.1663',
    'epochs_per_second: 0.0017',
    'epochs_per_min: 0.1002',
    'epochs_per_hr: 6.0123',
    'batches_per_epoch: 2916',
    'samples_per_epoch: 4.199e+05',
    'seconds_per_batch: 4.8700',
    'batches_per_second: 0.2053',
    'batches_per_min: 12.3203',
    'batches_per_hr: 739.2197',
    'samples_per_second: 29.5688',
    'samples_per_min: 1774.1273',
    'samples_per_hr: 1.064e+05',
    'batch_size: 144'
]

### Experiment #7

- Using Accumulate_grad_batches=2

Started: 5:45 AM - 2022-03-23  
Ended:   x:xx AM - 2022-03-23  

In [9]:
batches_per_second = (1/4.81)
batches_per_epoch = 1458
batch_size=144

print(f"Experiment #7: batch_size={batch_size}, num_processes=4, num_devices=2")
print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #7: batch_size=144, num_processes=4, num_devices=2
Using 50% of samples


[
    'seconds_per_epoch: 303.11850311850316',
    'min_per_epoch: 5.0520',
    'hrs_per_epoch: 0.0842',
    'epochs_per_second: 0.0033',
    'epochs_per_min: 0.1979',
    'epochs_per_hr: 11.8765',
    'batches_per_epoch: 1458',
    'samples_per_epoch: 2.1e+05',
    'seconds_per_batch: 4.8100',
    'batches_per_second: 0.2079',
    'batches_per_min: 12.4740',
    'batches_per_hr: 748.4407',
    'samples_per_second: 29.9376',
    'samples_per_min: 1796.2578',
    'samples_per_hr: 1.078e+05',
    'batch_size: 144'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 606.2370062370063',
    'min_per_epoch: 10.1040',
    'hrs_per_epoch: 0.1684',
    'epochs_per_second: 0.0016',
    'epochs_per_min: 0.0990',
    'epochs_per_hr: 5.9383',
    'batches_per_epoch: 2916',
    'samples_per_epoch: 4.199e+05',
    'seconds_per_batch: 4.8100',
    'batches_per_second: 0.2079',
    'batches_per_min: 12.4740',
    'batches_per_hr: 748.4407',
    'samples_per_second: 29.9376',
    'samples_per_min: 1796.2578',
    'samples_per_hr: 1.078e+05',
    'batch_size: 144'
]

### Experiment #8

- Using Accumulate_grad_batches=2
- lr=1e-2

- Removed base_callbacks.yaml:
        -train.callbacks.lr_monitor \
        -train.callbacks.early_stopping \
        -train.callbacks.model_checkpoint


Started: 9:00 AM - 2022-03-23  
Ended:   x:xx AM - 2022-03-23  

In [10]:
batches_per_second = (1/ 4.26)
batches_per_epoch = 229
batch_size=128

print(f"Experiment #8: batch_size={batch_size}, num_processes=4, num_devices=2")


print("Using 1% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)


print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*50,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*100,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #8: batch_size=128, num_processes=4, num_devices=2
Using 1% of samples


[
    'seconds_per_epoch: 53.755868544600936',
    'min_per_epoch: 0.8959',
    'hrs_per_epoch: 0.0149',
    'epochs_per_second: 0.0186',
    'epochs_per_min: 1.1162',
    'epochs_per_hr: 66.9694',
    'batches_per_epoch: 229',
    'samples_per_epoch: 2.931e+04',
    'seconds_per_batch: 4.2600',
    'batches_per_second: 0.2347',
    'batches_per_min: 14.0845',
    'batches_per_hr: 845.0704',
    'samples_per_second: 30.0469',
    'samples_per_min: 1802.8169',
    'samples_per_hr: 1.082e+05',
    'batch_size: 128'
]

Using 50% of samples


[
    'seconds_per_epoch: 2,687.7934272300467',
    'min_per_epoch: 44.7966',
    'hrs_per_epoch: 0.7466',
    'epochs_per_second: 0.0004',
    'epochs_per_min: 0.0223',
    'epochs_per_hr: 1.3394',
    'batches_per_epoch: 1.145e+04',
    'samples_per_epoch: 1.466e+06',
    'seconds_per_batch: 4.2600',
    'batches_per_second: 0.2347',
    'batches_per_min: 14.0845',
    'batches_per_hr: 845.0704',
    'samples_per_second: 30.0469',
    'samples_per_min: 1802.8169',
    'samples_per_hr: 1.082e+05',
    'batch_size: 128'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 5,375.5868544600935',
    'min_per_epoch: 89.5931',
    'hrs_per_epoch: 1.4932',
    'epochs_per_second: 0.0002',
    'epochs_per_min: 0.0112',
    'epochs_per_hr: 0.6697',
    'batches_per_epoch: 2.29e+04',
    'samples_per_epoch: 2.931e+06',
    'seconds_per_batch: 4.2600',
    'batches_per_second: 0.2347',
    'batches_per_min: 14.0845',
    'batches_per_hr: 845.0704',
    'samples_per_second: 30.0469',
    'samples_per_min: 1802.8169',
    'samples_per_hr: 1.082e+05',
    'batch_size: 128'
]

### Experiment #11

- Using Accumulate_grad_batches=1
- lr=0.5e-3
- freeze_backbone_up_to=-4


Started: 12:25 PM - 2022-03-23  
Ended:   2:55 PM - 2022-03-23  

In [11]:
batches_per_second = (1/ 4.53)
batches_per_epoch = 3282
batch_size=128

print(f"Experiment #11: batch_size={batch_size}, num_processes=4, num_devices=2")


print("Using 1% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)


print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*50,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch*100,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #11: batch_size=128, num_processes=4, num_devices=2
Using 1% of samples


[
    'seconds_per_epoch: 724.5033112582781',
    'min_per_epoch: 12.0751',
    'hrs_per_epoch: 0.2013',
    'epochs_per_second: 0.0014',
    'epochs_per_min: 0.0828',
    'epochs_per_hr: 4.9689',
    'batches_per_epoch: 3282',
    'samples_per_epoch: 4.201e+05',
    'seconds_per_batch: 4.5300',
    'batches_per_second: 0.2208',
    'batches_per_min: 13.2450',
    'batches_per_hr: 794.7020',
    'samples_per_second: 28.2561',
    'samples_per_min: 1695.3642',
    'samples_per_hr: 1.017e+05',
    'batch_size: 128'
]

Using 50% of samples


[
    'seconds_per_epoch: 36,225.165562913906',
    'min_per_epoch: 603.7528',
    'hrs_per_epoch: 10.0625',
    'epochs_per_second: 0.0000',
    'epochs_per_min: 0.0017',
    'epochs_per_hr: 0.0994',
    'batches_per_epoch: 1.641e+05',
    'samples_per_epoch: 2.1e+07',
    'seconds_per_batch: 4.5300',
    'batches_per_second: 0.2208',
    'batches_per_min: 13.2450',
    'batches_per_hr: 794.7020',
    'samples_per_second: 28.2561',
    'samples_per_min: 1695.3642',
    'samples_per_hr: 1.017e+05',
    'batch_size: 128'
]

(Extrapolated prediction) Using 100% of samples


[
    'seconds_per_epoch: 72,450.33112582781',
    'min_per_epoch: 1207.5055',
    'hrs_per_epoch: 20.1251',
    'epochs_per_second: 0.0000',
    'epochs_per_min: 0.0008',
    'epochs_per_hr: 0.0497',
    'batches_per_epoch: 3.282e+05',
    'samples_per_epoch: 4.201e+07',
    'seconds_per_batch: 4.5300',
    'batches_per_second: 0.2208',
    'batches_per_min: 13.2450',
    'batches_per_hr: 794.7020',
    'samples_per_second: 28.2561',
    'samples_per_min: 1695.3642',
    'samples_per_hr: 1.017e+05',
    'batch_size: 128'
]

### Experiment #12

- Using Accumulate_grad_batches=1
- lr=1e-2
- freeze_backbone_up_to=-4
- batch_size=128
- preprocess_size=256
- resolution=224


Started: 3:00 PM - 2022-03-23  
Ended:   x:xx AM - 2022-03-23  

In [12]:
batches_per_second = (1/ 2.58)
batches_per_epoch = 3282
batch_size=128

print(f"Experiment #8: batch_size={batch_size}, num_processes=4, num_devices=2")


print("Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)


print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch/2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 1% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch/100,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #8: batch_size=128, num_processes=4, num_devices=2
Using 100% of samples


[
    'seconds_per_epoch: 1,272.093023255814',
    'min_per_epoch: 21.2016',
    'hrs_per_epoch: 0.3534',
    'epochs_per_second: 0.0008',
    'epochs_per_min: 0.0472',
    'epochs_per_hr: 2.8300',
    'batches_per_epoch: 3282',
    'samples_per_epoch: 4.201e+05',
    'seconds_per_batch: 2.5800',
    'batches_per_second: 0.3876',
    'batches_per_min: 23.2558',
    'batches_per_hr: 1395.3488',
    'samples_per_second: 49.6124',
    'samples_per_min: 2976.7442',
    'samples_per_hr: 1.786e+05',
    'batch_size: 128'
]

Using 50% of samples


[
    'seconds_per_epoch: 636.046511627907',
    'min_per_epoch: 10.6008',
    'hrs_per_epoch: 0.1767',
    'epochs_per_second: 0.0016',
    'epochs_per_min: 0.0943',
    'epochs_per_hr: 5.6600',
    'batches_per_epoch: 1641',
    'samples_per_epoch: 2.1e+05',
    'seconds_per_batch: 2.5800',
    'batches_per_second: 0.3876',
    'batches_per_min: 23.2558',
    'batches_per_hr: 1395.3488',
    'samples_per_second: 49.6124',
    'samples_per_min: 2976.7442',
    'samples_per_hr: 1.786e+05',
    'batch_size: 128'
]

(Extrapolated prediction) Using 1% of samples


[
    'seconds_per_epoch: 12.720930232558139',
    'min_per_epoch: 0.2120',
    'hrs_per_epoch: 0.0035',
    'epochs_per_second: 0.0786',
    'epochs_per_min: 4.7166',
    'epochs_per_hr: 282.9982',
    'batches_per_epoch: 32.82',
    'samples_per_epoch: 4201',
    'seconds_per_batch: 2.5800',
    'batches_per_second: 0.3876',
    'batches_per_min: 23.2558',
    'batches_per_hr: 1395.3488',
    'samples_per_second: 49.6124',
    'samples_per_min: 2976.7442',
    'samples_per_hr: 1.786e+05',
    'batch_size: 128'
]

### Experiment #13
(Running in parallel to #12, since 4 GPUs just opened up.Tried doubling the scaling of the lr to accomodate the doubling of the # of GPUs


- Increased num_devices from 2->4
- Using Accumulate_grad_batches=1
- lr=2e-2
- freeze_backbone_up_to=-4
- batch_size=128
- preprocess_size=256
- resolution=224


Started: 3:52 PM - 2022-03-23  
Ended:   x:xx AM - 2022-03-23  

In [13]:
batches_per_second = (1/ 4.2)
batches_per_epoch = 1642
batch_size=128

print(f"Experiment #8: batch_size={batch_size}, num_processes=4, num_devices=2")


print("Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)


print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch/2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 1% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch/100,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #8: batch_size=128, num_processes=4, num_devices=2
Using 100% of samples


[
    'seconds_per_epoch: 390.9523809523809',
    'min_per_epoch: 6.5159',
    'hrs_per_epoch: 0.1086',
    'epochs_per_second: 0.0026',
    'epochs_per_min: 0.1535',
    'epochs_per_hr: 9.2083',
    'batches_per_epoch: 1642',
    'samples_per_epoch: 2.102e+05',
    'seconds_per_batch: 4.2000',
    'batches_per_second: 0.2381',
    'batches_per_min: 14.2857',
    'batches_per_hr: 857.1429',
    'samples_per_second: 30.4762',
    'samples_per_min: 1828.5714',
    'samples_per_hr: 1.097e+05',
    'batch_size: 128'
]

Using 50% of samples


[
    'seconds_per_epoch: 195.47619047619045',
    'min_per_epoch: 3.2579',
    'hrs_per_epoch: 0.0543',
    'epochs_per_second: 0.0051',
    'epochs_per_min: 0.3069',
    'epochs_per_hr: 18.4166',
    'batches_per_epoch: 821',
    'samples_per_epoch: 1.051e+05',
    'seconds_per_batch: 4.2000',
    'batches_per_second: 0.2381',
    'batches_per_min: 14.2857',
    'batches_per_hr: 857.1429',
    'samples_per_second: 30.4762',
    'samples_per_min: 1828.5714',
    'samples_per_hr: 1.097e+05',
    'batch_size: 128'
]

(Extrapolated prediction) Using 1% of samples


[
    'seconds_per_epoch: 3.90952380952381',
    'min_per_epoch: 0.0652',
    'hrs_per_epoch: 0.0011',
    'epochs_per_second: 0.2558',
    'epochs_per_min: 15.3471',
    'epochs_per_hr: 920.8283',
    'batches_per_epoch: 16.42',
    'samples_per_epoch: 2102',
    'seconds_per_batch: 4.2000',
    'batches_per_second: 0.2381',
    'batches_per_min: 14.2857',
    'batches_per_hr: 857.1429',
    'samples_per_second: 30.4762',
    'samples_per_min: 1828.5714',
    'samples_per_hr: 1.097e+05',
    'batch_size: 128'
]

### Experiment #14


- Increased num_devices from 4
- Using Accumulate_grad_batches=2
- lr=1e-3
- freeze_backbone=False
- batch_size=64
- preprocess_size=256
- resolution=224


Started: 5:00 PM - 2022-03-23  
Ended:   x:xx AM - 2022-03-24

In [24]:
print(f"{1/((3282)/(90*60)):.2f}")

1.65


In [15]:
batches_per_second = (1/1.65 )
batches_per_epoch = 3282
batch_size=64

print(f"Experiment #8: batch_size={batch_size}, num_processes=4, num_devices=2")


print("Using 100% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)


print("Using 50% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch/2,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

print("(Extrapolated prediction) Using 1% of samples")
display_train_timing_info(batches_per_epoch=batches_per_epoch/100,
                          batches_per_second=batches_per_second,
                          batch_size=batch_size)

Experiment #8: batch_size=64, num_processes=4, num_devices=2
Using 100% of samples


[
    'seconds_per_epoch: 1,989.0909090909092',
    'min_per_epoch: 33.1515',
    'hrs_per_epoch: 0.5525',
    'epochs_per_second: 0.0005',
    'epochs_per_min: 0.0302',
    'epochs_per_hr: 1.8099',
    'batches_per_epoch: 3282',
    'samples_per_epoch: 2.1e+05',
    'seconds_per_batch: 1.6500',
    'batches_per_second: 0.6061',
    'batches_per_min: 36.3636',
    'batches_per_hr: 2181.8182',
    'samples_per_second: 38.7879',
    'samples_per_min: 2327.2727',
    'samples_per_hr: 1.396e+05',
    'batch_size: 64'
]

Using 50% of samples


[
    'seconds_per_epoch: 994.5454545454546',
    'min_per_epoch: 16.5758',
    'hrs_per_epoch: 0.2763',
    'epochs_per_second: 0.0010',
    'epochs_per_min: 0.0603',
    'epochs_per_hr: 3.6197',
    'batches_per_epoch: 1641',
    'samples_per_epoch: 1.05e+05',
    'seconds_per_batch: 1.6500',
    'batches_per_second: 0.6061',
    'batches_per_min: 36.3636',
    'batches_per_hr: 2181.8182',
    'samples_per_second: 38.7879',
    'samples_per_min: 2327.2727',
    'samples_per_hr: 1.396e+05',
    'batch_size: 64'
]

(Extrapolated prediction) Using 1% of samples


[
    'seconds_per_epoch: 19.89090909090909',
    'min_per_epoch: 0.3315',
    'hrs_per_epoch: 0.0055',
    'epochs_per_second: 0.0503',
    'epochs_per_min: 3.0165',
    'epochs_per_hr: 180.9872',
    'batches_per_epoch: 32.82',
    'samples_per_epoch: 2100',
    'seconds_per_batch: 1.6500',
    'batches_per_second: 0.6061',
    'batches_per_min: 36.3636',
    'batches_per_hr: 2181.8182',
    'samples_per_second: 38.7879',
    'samples_per_min: 2327.2727',
    'samples_per_hr: 1.396e+05',
    'batch_size: 64'
]

In [125]:
229/16/60

0.23854166666666668

In [14]:
27.96*2

55.92